In [7]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch


In [8]:
class HAM10000Dataset(Dataset):
    def __init__(self, csv_file, img_dir, file_prefix = "", transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.file_prefix = file_prefix
        self.transform = transform
        self.classes = sorted(self.data.iloc[:, 2].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.file_prefix + self.data.iloc[idx, 1] + '.jpg')
        image = Image.open(img_name)
        label = self.data.iloc[idx, 2]
        label_idx = self.class_to_idx[label]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label_idx, dtype=torch.long)

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = HAM10000Dataset(csv_file='data/archive/HAM10000_metadata.csv', img_dir='data/hair_removed/', file_prefix="nohair_", transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [10]:
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights

def create_model(num_classes=7):
    # todo: what weights to use?
    model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1)

    # todo: modify the classifier?
    model.classifier[2] = torch.nn.Linear(768, num_classes)

    return model

num_classes = len(dataset.classes)
model = create_model(num_classes=num_classes)

if torch.cuda.is_available():
    model = model.cuda()

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        if isinstance(labels, tuple):
            labels = labels[0]

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_dataloader)}")

    # Validation step
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()
            if isinstance(labels, tuple):
                labels = labels[0]

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Validation Loss: {val_loss/len(test_dataloader)}, Accuracy: {100.*correct/total}%")

print("Training completed.")

Epoch 1/10, Training Loss: 0.7056400172738915
Validation Loss: 0.5218399500563031, Accuracy: 81.47778332501248%
Epoch 2/10, Training Loss: 0.4241863518240917
Validation Loss: 0.5172214529344014, Accuracy: 79.73040439340988%
Epoch 3/10, Training Loss: 0.239330586654494
Validation Loss: 0.44144364682927967, Accuracy: 85.57164253619571%
Epoch 4/10, Training Loss: 0.11539133306429443
Validation Loss: 0.5863603198575595, Accuracy: 84.47329006490264%
Epoch 5/10, Training Loss: 0.074382930044159
Validation Loss: 0.6233507359311694, Accuracy: 83.82426360459311%
Epoch 6/10, Training Loss: 0.040729698802036536
Validation Loss: 0.5935453451755974, Accuracy: 85.87119321018473%
Epoch 7/10, Training Loss: 0.038065784589009014
Validation Loss: 0.6753175209261595, Accuracy: 85.12231652521218%
Epoch 8/10, Training Loss: 0.04428494533484958
Validation Loss: 0.6633431913833769, Accuracy: 85.12231652521218%
Epoch 9/10, Training Loss: 0.0313248459024615
Validation Loss: 0.6457099918098677, Accuracy: 84.972

In [14]:
model.eval()
correct = 0
total = 0

idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

with torch.no_grad():
    for images, labels in test_dataloader:
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Convert numeric predictions back to string labels if needed
        predicted_labels = [idx_to_class[idx.item()] for idx in predicted]
        true_labels = [idx_to_class[idx.item()] for idx in labels]

        print(f"Predicted: {predicted_labels}")
        print(f"Actual: {true_labels}")

print(f'Accuracy: {100 * correct / total}%')

Predicted: ['nv', 'nv', 'nv', 'nv', 'akiec', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'mel', 'mel', 'nv', 'nv', 'nv', 'nv', 'akiec', 'mel', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'bkl', 'nv', 'nv', 'mel', 'nv']
Actual: ['nv', 'nv', 'nv', 'nv', 'bcc', 'nv', 'nv', 'nv', 'mel', 'bkl', 'nv', 'bcc', 'nv', 'nv', 'nv', 'nv', 'nv', 'akiec', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'nv', 'bkl', 'nv', 'nv', 'mel', 'nv']
Predicted: ['df', 'nv', 'nv', 'akiec', 'nv', 'mel', 'nv', 'vasc', 'bkl', 'nv', 'mel', 'nv', 'nv', 'nv', 'bcc', 'nv', 'nv', 'nv', 'bcc', 'nv', 'vasc', 'nv', 'nv', 'nv', 'nv', 'nv', 'bkl', 'nv', 'nv', 'nv', 'mel', 'mel']
Actual: ['bcc', 'nv', 'nv', 'bcc', 'nv', 'mel', 'nv', 'vasc', 'bkl', 'nv', 'mel', 'nv', 'nv', 'nv', 'bcc', 'nv', 'nv', 'mel', 'bcc', 'nv', 'vasc', 'mel', 'nv', 'nv', 'nv', 'nv', 'bkl', 'nv', 'nv', 'nv', 'mel', 'mel']
Predicted: ['nv', 'nv', 'nv', 'df', 'nv', 'bkl', 'nv', 'nv', 'nv', 'mel', 'nv', 'nv', 'nv', 'akiec', 'mel', 'nv', 'nv', 'bkl', 'mel', 'nv',